In [1]:
import Random

In [93]:
## OPEN - MULTILINE

function mcs_open(vehicles, velocities, max_velocity, min_velocity, p)
    """
    Perform 1 MCS. Velocity update happens in parallel for every car. 
    """
    
    update_velocities_open(vehicles, velocities, max_velocity, min_velocity, p)
    
    print_state(vehicles, velocities)
    
    update_positions_open(vehicles, velocities)
    
end

function update_velocities_open(vehicles, velocities, max_velocity, min_velocity, p)
    """
    Update velocities of vehicles in open system. Performed in-place.
    At the beginning next_vehicle points to empty space, later to vehicle in front.
    1. If next_vehicle is not in range of current vehicle's move, try to accelerate.
       If next_vehicle is in range of move, slow down.
    2. Random slow down event
    """
    
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    
    # Holds index at which collision occurs. At the beginning we take impossible index.
    for j = 1:lines
        next_vehicle = length + max_velocity + 1
    
        for i = length:-1:1
            # if vehicle exists on position
            if vehicles[j,i] != 0

                # if no collision
                if next_vehicle > i + velocities[j,i]

                    accelerate(velocities, i, j, next_vehicle, max_velocity)

                    slow_down_random(velocities, i, j, p)

                else
                    
                    ## TU WYPRZEDZANKO
                    
                    slow_down_car(velocities, i, j, next_vehicle)

                    slow_down_random(velocities, i, j, p)

                end

                next_vehicle = i
            end
        end
    end
end

function update_positions_open(vehicles, velocities)
    """
    Update positions in open system given current positions and velocities. Performed in-place.
    If vehicle at one of the last 6 places, delete it.
    If vehicle has speed 0, leave it.
    If not, move vehicle to new index = (its index + its velocity).
    Crashes are taken care of in update_velocity, no need to check them here
    """
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    for j = 1:lines
        for i = length:-1:1
            if vehicles[j,i] != 0 && velocities[j,i] != 0

                # Check if move out of bounds
                if i > length - 6
                    vehicles[j,i] = 0
                    velocities[j,i] = 0
                else
                    # Update new position in vehicles and velocities
                    # Clear old position

                    vehicles[j, (i + velocities[j,i])] = 1
                    velocities[j, (i + velocities[j,i])] = velocities[j,i]
                    vehicles[j,i] = 0
                    velocities[j,i] = 0
                end
            end
        end
    end
end

update_positions_open (generic function with 1 method)

In [100]:
# closed boundary - multiline

function mcs_closed(vehicles, velocities, max_velocity, min_velocity, p)
    """
    Performs 1 MCS. Velocity updates happen in parallel for every car.
    """
    
    update_velocities_closed(vehicles, velocities, max_velocity, min_velocity, p)
    
    print_state(vehicles, velocities)
    
    update_positions_closed(vehicles, velocities)
    
end

function update_velocities_closed(vehicles, velocities, max_velocity, min_velocity, p)
    """
    """
    
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    
    for j in 1:lines
        next_vehicle = 0
        
        for i = 1:length
            if vehicles[j,i]!=0
                next_vehicle = i + length
            break
            end
        end
        for i = length:-1:1
                if vehicles[j, i]!=0
                    if next_vehicle > i + velocities[j, i]
                        accelerate(velocities, i, j, next_vehicle, max_velocity)
                        slow_down_random(velocities, i, j, p)
                    else
                        slow_down_car(velocities, i, j, next_vehicle)
                        slow_down_random(velocities, i, j, p)
                    end
            
                    next_vehicle = i
                end
         end
    end
end

function update_positions_closed(vehicles, velocities)
        """
        
        """
    length = size(velocities)[2]
    lines = size(velocities)[1]
    
    # If rightmost vehicle goes through periodic boundary, avoid updating it second time by saving it's new position
    # At the beginning avoid_pos holds value out of bounds
    
    avoid_pos=length+10
    
    for j = 1:lines
        for i = length:-1:1
            if (vehicles[j, i] != 0) && (velocities[j, i] != 0) && (i !=avoid_pos)
                if (i+velocities[j, i]) > length
                    
                    new_pos=(i+velocities[j, i])%length
                    vehicles[j, new_pos] = 1
                    velocities[j, new_pos] = velocities[j, i]
                    avoid_pos=new_pos
                    vehicles[j, i] = 0
                    velocities[j, i] = 0
                    
                else
                    vehicles[j, (i+velocities[j, i])] = 1
                    velocities[j, (i+velocities[j, i])] = velocities[j, i]
                    vehicles[j, i]=0
                    velocities[j, i]=0
                end
            end
        end
    end
end
            

update_positions_closed (generic function with 1 method)

In [42]:
# try to accelerate
function accelerate(velocities, pos, row, next_vehicle, max_velocity)
    """
    If velocity is smaller than max_velocity chceck if collision occurs.
    If no collision => accelerate
    """
    if velocities[row,pos] < max_velocity
        if pos + velocities[row,pos] + 1 < next_vehicle
            velocities[row,pos] += 1
        end
    end
end


# slow down due to next car
function slow_down_car(velocities, pos, row,  next_vehicle)
    """
    Slow down to land on position (next_vehicle - 1) after move.
    """
    velocities[row,pos] = next_vehicle - pos - 1
end


# slow down randomly
function slow_down_random(velocities, pos, row, p)
    """
    Random event of slowing down with probability p. 
    Velocity cannot drop below 0.
    """
    if Random.rand() < p
        if velocities[row,pos] > 0
            velocities[row,pos] -= 1
        end
    end
end

slow_down_random (generic function with 2 methods)

In [102]:
##MAIN PROGRAM
length = 80
lines = 2
p = 0.15
max_velocity = 5 
min_velocity = 0

0

In [105]:
function load_scenario(vehicles, velocities, no)
    if no == 1
        vehicles[1, 1]=1
        velocities[1, 1]=5
        vehicles[2, 3]=1
        velocities[2,3]=0
    end
end

load_scenario (generic function with 1 method)

In [63]:
function print_state(vehicles, velocities)
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    for i = 1:lines
        str = ""
        for j = 1:length
            if vehicles[i,j] == 0
                str = str * "."
            else
                str = str * string(velocities[i,j])
            end
        end
        println(str)
    end
    str=""
    for i = 1:length
        str=str*"="
    end
    println(str)
end

print_state (generic function with 1 method)

In [94]:
#open boundary - multiline

velocities = fill(0, (lines,length))
vehicles = fill(0, (lines,length))

no_steps = 40

for step = 1:no_steps
    for j in 1:lines
        if vehicles[j,1] == 0
            vehicles[j,1] = 1
            velocities[j,1] = 0
        end
    end 
    mcs_open(vehicles, velocities, max_velocity, min_velocity, p)
end

print_state(vehicles, velocities)

1...............................................................................
1...............................................................................
02..............................................................................
02..............................................................................
1..3............................................................................
1..3............................................................................
02....4.........................................................................
02....4.........................................................................
1..3......5.....................................................................
1..3......5.....................................................................
02....3........5................................................................
01....4........5................................................................
1..3.....4..........5.......

1..3....3...4.............5......5...........5..............4............5......
02....4........5.......4.......4......4............3....5.............5.........
02....4....3....4..............5......5...........5.............4.............5.
0..2......4.........5......4.......4......4...........3......5.............5....
0..2......4...3.....4...............5......5...........5............4...........


In [ ]:


# closed boundary -  multiline

velocities = fill(0, (lines, length))
vehicles = fill(0, (lines, length))

no_cars=60
no_steps=30

vehicles[1, 1:no_cars] .=1
Random.shuffle!(vehicles)

for step = 1:no_steps
    mcs_closed(vehicles, velocities, max_velocity, min_velocity, p)
end


In [ ]:
# closed boundary - testing


In [107]:
velocities = fill(0, (lines, length))
vehicles = fill(0, (lines, length))

no_steps=30
scenario_no=1
load_scenario(vehicles, velocities, scenario_no)

for step = 1:no_steps
    mcs_closed(vehicles, velocities, max_velocity, min_velocity, p)
end


5...............................................................................
..0.............................................................................
.....5..........................................................................
..1.............................................................................
..........5.....................................................................
...1............................................................................
...............5................................................................
....2...........................................................................
....................5...........................................................
......3.........................................................................
.........................5......................................................
.........4......................................................................
............................